<a href="https://colab.research.google.com/github/Nacho2904/pasantiacsc/blob/main/agosto30.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>